In [65]:
import networkx as nx
# create a DAG
g = nx.DiGraph()
# add nl[j]s and edges to the DAG
g.add_edges_from([("A", "D"), ("B", "D"), ("B", "E"), ("B", "G"), ("C", "E"),("C", "F"), ("D", "I"), ("E", "G"),("E", "H"), ("F", "H"), ("G", "I"), ("G","J"), ("H", "J")])
nx.set_node_attributes(g, {"A": "PI", "B": "PI", "C":"PI", "D":"AND2", "E":"XOR2", "F":"INV", "G":"AND2", "H":"AND2", "I":"OR2", "J":"OR2"},name="gateType")
print(g.nodes(data=True))
# sort the nodes in topological order
nl = list(nx.topological_sort(g))
print('Nodes in topological order',nl)
# find children of a node
print('Successors of node B', list(g.successors("B")))
# find parents of a node
print('Parents of node H', list(g.predecessors("H")))
['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

[('A', {'gateType': 'PI'}), ('D', {'gateType': 'AND2'}), ('B', {'gateType': 'PI'}), ('E', {'gateType': 'XOR2'}), ('G', {'gateType': 'AND2'}), ('C', {'gateType': 'PI'}), ('F', {'gateType': 'INV'}), ('I', {'gateType': 'OR2'}), ('H', {'gateType': 'AND2'}), ('J', {'gateType': 'OR2'})]


TypeError: 'NodeDataView' object is not callable

# Getting the input from the netlist file

In [27]:

# The program will throw in an error if there isn't exactly 2 arguments in the commandline.
# if len(argv)!=2:
# 	print("Please provide the correct 2 arguments in the commandline.")
# 	exit()
import networkx as nx

class Component:
	def __init__(self,name,g_type,input,output):
		self.name = name
		self.g_type = g_type
		self.input = input
		self.output = output
l = []
	# Opening the file mentioned in the commandline.
with open("c8.net") as f:
    lines = f.readlines()
    for line in lines:
        name,g_type,*input= line.split()
        output = line.split()[-1]
        Gate = Component(name,g_type,input,output)
        l.append(Gate)

edges = []
g = nx.DiGraph()
for gate in l:
    if len(gate.input) == 3:
        edges.append((gate.input[0],gate.output))
        edges.append((gate.input[1],gate.output))
    else:
        edges.append((gate.input[0],gate.output))

g.add_edges_from(edges)
value = []
with open("c8.inputs") as file:
    lineIn = file.readlines()
    pi = lineIn[0].split()
    for i in range(len(lineIn)-1):
        value.append({})
        for j in range(len(pi)):
            value[i].update({lineIn[0].split()[j]:lineIn[i+1].split()[j]})
value1 = value.copy()
# print(value)
gate_type = {}

for node in pi:
     gate_type.update({node:"PI"})

for node in l:
     gate_type.update({node.output:node.g_type})
nx.set_node_attributes(g,gate_type,name = "gateType")

nl = list(nx.topological_sort(g))
print(g)
print('Nodes in topological order',nl)

Graph = g.nodes(data=True)

typemap= {}
for item in Graph:
    typemap.update({item[0]:item[1]["gateType"]})
print(value[0])

DiGraph with 135 nodes and 206 edges
Nodes in topological order ['q', 'h', 'g', 'f', 'bn0', 'e', 'an0', 'd', 'r', 'cn0', 'y', 'z', 'c', 'b', 'a', 'v', 'w', 'x', 'i', 'l', 'k', 'm', 'n', 'j', 'p', 'u', 'o', 's', 'n_1', 'n_0', 'dummy_11', 'dummy_32', 'dummy_37', 'n_20', 'ln0', 'dummy_28', 'dummy_12', 'dummy_30', 'n_43', 'dummy_39', 'n_2', 'dummy_21', 'dummy_26', 'dummy_24', 'n_31', 'dummy_34', 'dummy_36', 'n_5', 'n_16', 'n_60', 'n_7', 'dummy_25', 'dummy_27', 'dummy_29', 'dummy_31', 'dummy_33', 'dummy_35', 'dummy_38', 'dummy_40', 'dummy_23', 'n_6', 'n_25', 'n_23', 'n_17', 'n_14', 'n_12', 'n_11', 'n_10', 'n_9', 'n_8', 'dummy_1', 'dummy_3', 'dummy_5', 'dummy_7', 'dummy_10', 'dummy_15', 'dummy_18', 'dummy_20', 'dummy_22', 'gn0', 'fn0', 'hn0', 'in0', 'kn0', 'dn0', 'jn0', 'en0', 'n_24', 'n_27', 'n_26', 'n_32', 'n_28', 'dummy_19', 'dummy_17', 'dummy_16', 'n_35', 'n_30', 'n_33', 'n_29', 'n_34', 'n_37', 'n_36', 'n_44', 'dummy_14', 'dummy_13', 'mn0', 'nn0', 'dummy_9', 'dummy_8', 'n_48', 'n_42', 'n

# Solving using the topological order

In [28]:

for i in range(len(lineIn)-1):   
   
   for j in range(len(pi),len(nl)):
      p = list(g.predecessors(nl[j]))
      out = 0 
      
      if typemap[nl[j]] == "nand2":
         a =  int(value[i][p[0]]) * int(value[i][p[1]])
         if a == 0 :
            out = 1
            
      if typemap[nl[j]] == "and2":
         out =  int(value[i][p[0]]) * int(value[i][p[1]])

      if typemap[nl[j]] == "nor2":
         if int(value[i][p[0]]) + int(value[i][p[1]]) < 1:
            out = 1
      
      if typemap[nl[j]] == "or2":
         if int(value[i][p[0]]) + int(value[i][p[1]]) > 1:
            out = 1

      if typemap[nl[j]] == "inv":
         if int(value[i][p[0]]) == 0:
            out = 1

      if typemap[nl[j]] == "xnor2":
         if int(value[i][p[0]]) == int(value[i][p[1]]):
            out = 1

      if typemap[nl[j]] == "or2":
         if int(value[i][p[0]]) != int(value[i][p[1]]):
            out = 1
      value[i].update({nl[j]:out})
print(value[0])
           

{'a': '0', 'b': '1', 'c': '0', 'd': '0', 'e': '0', 'f': '1', 'g': '0', 'h': '0', 'i': '1', 'j': '1', 'k': '0', 'l': '0', 'm': '1', 'n': '0', 'o': '1', 'p': '1', 'q': '0', 'r': '1', 's': '1', 'u': '0', 'v': '0', 'w': '0', 'x': '1', 'y': '1', 'z': '1', 'an0': '1', 'bn0': '1', 'cn0': '0', 'n_1': 1, 'n_0': 0, 'dummy_11': 0, 'dummy_32': 1, 'dummy_37': 1, 'n_20': 1, 'ln0': 0, 'dummy_28': 1, 'dummy_12': 1, 'dummy_30': 1, 'n_43': 0, 'dummy_39': 0, 'n_2': 1, 'dummy_21': 0, 'dummy_26': 0, 'dummy_24': 1, 'n_31': 0, 'dummy_34': 0, 'dummy_36': 0, 'n_5': 0, 'n_16': 0, 'n_60': 1, 'n_7': 1, 'dummy_25': 1, 'dummy_27': 1, 'dummy_29': 1, 'dummy_31': 1, 'dummy_33': 1, 'dummy_35': 1, 'dummy_38': 1, 'dummy_40': 1, 'dummy_23': 1, 'n_6': 1, 'n_25': 1, 'n_23': 1, 'n_17': 1, 'n_14': 1, 'n_12': 1, 'n_11': 1, 'n_10': 1, 'n_9': 1, 'n_8': 1, 'dummy_1': 0, 'dummy_3': 0, 'dummy_5': 1, 'dummy_7': 0, 'dummy_10': 0, 'dummy_15': 0, 'dummy_18': 1, 'dummy_20': 0, 'dummy_22': 1, 'gn0': 0, 'fn0': 1, 'hn0': 0, 'in0': 0, 'kn0'

# Using the Event - driven method

In [29]:
from collections import deque
for i in range(len(lineIn)-1):
    Event = deque(nl[:len(pi)])

    while len(Event) !=0:
        e = Event.popleft()
        for thing in list(g.successors(e)):
            if all(elem in value1[i] for elem in list(g.predecessors(thing))):
                Event.extend(list(g.successors(e)))

        if e not in value1[i]:
            print(e)
            p = list(g.predecessors(e))
            out = 0
            
            if typemap[e] == "nand2":
                a =  int(value1[i][p[0]]) * int(value1[i][p[1]])
                if a == 0 :
                    out = 1
                    
            if typemap[e] == "and2":
                out =  int(value1[i][p[0]]) * int(value1[i][p[1]])

            if typemap[e] == "nor2":
                if int(value1[i][p[0]]) + int(value1[i][p[1]]) < 1:
                    out = 1
            
            if typemap[e] == "or2":
                if int(value1[i][p[0]]) + int(value1[i][p[1]]) > 1:
                    out = 1

            if typemap[e] == "inv":
                if int(value1[i][p[0]]) == 0:
                    out = 1

            if typemap[e] == "xnor2":
                if int(value1[i][p[0]]) == int(value1[i][p[1]]):
                    out = 1

            if typemap[e] == "or2":
                if int(value1[i][p[0]]) != int(value1[i][p[1]]):
                    out = 1
                    
            value1[i].update({e:out})

print(value1[0])

{'a': '0', 'b': '1', 'c': '0', 'd': '0', 'e': '0', 'f': '1', 'g': '0', 'h': '0', 'i': '1', 'j': '1', 'k': '0', 'l': '0', 'm': '1', 'n': '0', 'o': '1', 'p': '1', 'q': '0', 'r': '1', 's': '1', 'u': '0', 'v': '0', 'w': '0', 'x': '1', 'y': '1', 'z': '1', 'an0': '1', 'bn0': '1', 'cn0': '0', 'n_1': 1, 'n_0': 0, 'dummy_11': 0, 'dummy_32': 1, 'dummy_37': 1, 'n_20': 1, 'ln0': 0, 'dummy_28': 1, 'dummy_12': 1, 'dummy_30': 1, 'n_43': 0, 'dummy_39': 0, 'n_2': 1, 'dummy_21': 0, 'dummy_26': 0, 'dummy_24': 1, 'n_31': 0, 'dummy_34': 0, 'dummy_36': 0, 'n_5': 0, 'n_16': 0, 'n_60': 1, 'n_7': 1, 'dummy_25': 1, 'dummy_27': 1, 'dummy_29': 1, 'dummy_31': 1, 'dummy_33': 1, 'dummy_35': 1, 'dummy_38': 1, 'dummy_40': 1, 'dummy_23': 1, 'n_6': 1, 'n_25': 1, 'n_23': 1, 'n_17': 1, 'n_14': 1, 'n_12': 1, 'n_11': 1, 'n_10': 1, 'n_9': 1, 'n_8': 1, 'dummy_1': 0, 'dummy_3': 0, 'dummy_5': 1, 'dummy_7': 0, 'dummy_10': 0, 'dummy_15': 0, 'dummy_18': 1, 'dummy_20': 0, 'dummy_22': 1, 'gn0': 0, 'fn0': 1, 'hn0': 0, 'in0': 0, 'kn0'

In [30]:
for i in range(len(lineIn)-1):
    if set(value[i].items()) ^ set(value1[i].items()):
        print("There are differences")

In [13]:
from collections import deque

# Define the list N
N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define the size of the queue
n = 5

# Create a new deque object and add the first n values of N to the queue
queue = deque(N[:n])
l =[1,2,3]
queue.extend(l)
# Print the initial queue
print(queue)
if l in N:
    print("yes")

deque([1, 2, 3, 4, 5, 1, 2, 3])
